In [2]:
import google.colab
google.colab.drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from pathlib import Path
import re
from tqdm.notebook import tqdm
import numpy as np
import numpy as np
import glob

In [4]:
path_folder = Path('/content/drive/MyDrive/shakiba')
name_data = 'tweets_2015_2019_cleaned_EN.csv'
tweet_data = pd.read_csv(path_folder/name_data, usecols = ['timestamp'])
tweet_data['timestamp'] = pd.to_datetime(tweet_data['timestamp'],)
length  = int(len(tweet_data)/3)

In [15]:
numpy_path = glob.glob('/content/drive/MyDrive/shakiba/*finall.npy')
numpy_path.sort()

all_score = []
for i,path in enumerate(numpy_path):
  score_sentimen = np.load(path)
  score_sentimen[:,0] = score_sentimen[:,0] + i * length
  all_score.append(score_sentimen)

scores_sentimen = np.concat(all_score, axis=0)
data_sentiment = pd.DataFrame(scores_sentimen, columns=['index', 'positive', 'negetive', 'neutral'])
data_sentiment['index'] = data_sentiment['index'].astype('int')
data_sentiment.set_index('index')

,positive,negetive,neutral
index,,,
0,0.075139,0.751241,0.173620
1,0.153980,0.745753,0.100268
2,0.147979,0.293123,0.558899
3,0.070246,0.517064,0.412691
4,0.160777,0.555255,0.283969
...,...,...,...
8691057,0.082269,0.514000,0.403731
8691058,0.407545,0.454205,0.138250
8691059,0.068729,0.526453,0.404819


In [6]:
join_data = data_sentiment.join(tweet_data, how='inner').set_index('index')
join_data.groupby(join_data['timestamp'].dt.date).mean()
join_data = join_data.groupby(join_data['timestamp'].dt.date).mean().drop(['timestamp'], axis=1)

In [7]:
join_data

,positive,negetive,neutral
timestamp,,,
2015-01-01,0.116094,0.789252,0.094654
2015-01-02,0.113846,0.795338,0.090816
2015-01-03,0.114822,0.795150,0.090027
2015-01-04,0.114798,0.790363,0.094839
2015-01-05,0.117028,0.790906,0.092066
...,...,...,...
2019-11-19,0.210475,0.532893,0.256632
2019-11-20,0.223350,0.554299,0.222351
2019-11-21,0.306655,0.399242,0.294104


In [16]:
path_folder = Path('/content/drive/MyDrive/shakiba')
name_data = 'BTC_USD.csv'
bit_data = pd.read_csv(path_folder/name_data)

In [17]:
bit_data['Date'] = pd.to_datetime(bit_data['Date']).dt.date
bit_data.set_index('Date')

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2025-01-26,105000.0,104700.0,105440.0,104510.0,0.11K,0.29%
2025-01-25,104700.0,104800.0,105260.0,104100.0,0.14K,-0.10%
2025-01-24,104800.0,103930.0,107030.0,102720.0,0.52K,0.84%
2025-01-23,103930.0,103680.0,106780.0,101260.0,1.70K,0.24%
2025-01-22,103680.0,106150.0,106290.0,103400.0,0.51K,-2.33%
...,...,...,...,...,...,...
2015-01-05,273.2,261.0,276.5,260.5,2.68K,5.57%
2015-01-04,258.8,282.0,288.9,258.8,2.38K,-8.23%
2015-01-03,282.0,314.8,314.8,282.0,1.30K,-10.51%


In [18]:
bit_sentiment = pd.merge(left = join_data,
        right = bit_data,
        left_on = 'timestamp',
        right_on = 'Date',
        how = 'inner')
bit_sentiment

,positive,negetive,neutral,Date,Price,Open,High,Low,Vol.,Change %
0,0.116094,0.789252,0.094654,2015-01-01,313.9,317.5,317.5,312.9,0.93K,-0.99%
1,0.113846,0.795338,0.090816,2015-01-02,315.1,313.6,316.0,313.0,1.32K,0.40%
2,0.114822,0.795150,0.090027,2015-01-03,282.0,314.8,314.8,282.0,1.30K,-10.51%
3,0.114798,0.790363,0.094839,2015-01-04,258.8,282.0,288.9,258.8,2.38K,-8.23%
4,0.117028,0.790906,0.092066,2015-01-05,273.2,261.0,276.5,260.5,2.68K,5.57%
...,...,...,...,...,...,...,...,...,...,...
1781,0.210475,0.532893,0.256632,2019-11-19,8164.9,8219.5,8241.9,8055.2,3.70K,-0.66%
1782,0.223350,0.554299,0.222351,2019-11-20,8126.4,8164.9,8255.0,8081.0,2.06K,-0.47%
1783,0.306655,0.399242,0.294104,2019-11-21,7671.1,8126.4,8155.9,7510.1,9.89K,-5.60%
1784,0.202748,0.585070,0.212181,2019-11-22,7323.0,7672.5,7763.3,6820.1,19.37K,-4.54%


In [19]:
bit_sentiment.to_csv(path_folder/'bit_sen.csv', index=False)